In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import time
import os
import re
from itertools import chain
from transformers import BertTokenizer
PRETRAINED_MODEL_NAME = "bert-base-uncased" #加载BERT英文预训练模型，不区分大小写
print(torch.__version__)

1.7.1


In [3]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
vocab = tokenizer.vocab
print("dict size", len(vocab))

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

dict size 30522


In [4]:
from torch.utils.data import Dataset,random_split

In [5]:
TAG_RE = re.compile(r'<[^>]+>')
def preprocess_text(sen):
    # Removing html tags
    sentence = TAG_RE.sub('', sen)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [31]:
import logging
import random

import numpy as np
import torch
#加载fake news 数据集
"""
Load data from news sources.
Requires data from https://www.kaggle.com/c/fake-news/data to be saved to "./train.csv".

"""

# split data to 10 fold
fold_num = 10
data_file = 'train.csv'

import pandas as pd



f = pd.read_csv(data_file, encoding='UTF-8',usecols=['text', 'label'])
texts = f['text'].tolist()[:20761]
labels = f['label'].tolist()[:20761]
index = list(range(20761))
np.random.shuffle(index)
    
all_texts = []
all_labels = []
data=[]
for i in index:
    if labels[i] == 1:
        data.append([preprocess_text(str(texts[i])), 1])
    elif labels[i] == 0:
        data.append([preprocess_text(str(texts[i])), 0])

print(data[1])






['CHARLESTON C After three tours in Iraq and Afghanistan J Hardin wound up hiding from the world in backwoods cabin in North Carolina Divorced alcoholic and at times suicidal he had tried almost all the accepted treatments for stress disorder psychotherapy group therapy and nearly dozen different medications Nothing worked for me so put aside the idea that could get better said Mr Hardin just pretty much became hermit in my cabin and never went out Then in he joined small drug trial testing whether PTSD could be treated with MDMA the illegal party drug better known as Ecstasy It changed my life he said in recent interview in the bright airy living room of the suburban ranch house here where he now lives while going to college and working as an airplane mechanic It allowed me to see my trauma without fear or hesitation and finally process things and move forward Based on promising results like Mr Hardin the Food and Drug Administration gave permission Tuesday for Phase clinical trials o

In [32]:
import tensorflow as tf
class MyDataset(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  
        self.mode = mode
        self.df = data #its list [['text1',label],['text2',label],...]
        self.len = len(self.df)
        self.maxlen = 200 
        self.tokenizer = tokenizer  # we will use BERT tokenizer
    
    
    def __getitem__(self, idx):
        origin_text = self.df[idx][0]
        if self.mode == "test":
            text_a = self.df[idx][0]
            text_b = None  #for natural language inference
            #label_tensor = None #in our case, we have label
            label_id = self.df[idx][1]
            label_tensor = torch.tensor(label_id)
        else:     
            text_a = self.df[idx][0]
            text_b = None  #for natural language inference
            label_id = self.df[idx][1]
            label_tensor = torch.tensor(label_id)
            
        
       
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a[:self.maxlen] + ["[SEP]"]
        len_a = len(word_pieces)
        
        if text_b is not None:
            tokens_b = self.tokenizer.tokenize(text_b)
            word_pieces += tokens_b + ["[SEP]"]
            len_b = len(word_pieces) - len_a
               
       
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
       
        if text_b is None:
            segments_tensor = torch.tensor([1] * len_a,dtype=torch.long)
        elif text_b is not None:
            segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor, origin_text)
    
    def __len__(self):
        return self.len
    
# initialize Dataset
trainset = MyDataset("train", tokenizer=tokenizer)
# testset = MyDataset("test", tokenizer=tokenizer)


#split val from trainset
val_size = int(trainset.__len__()*0.06) 
trainset, valset = random_split(trainset,[trainset.__len__()-val_size,val_size])
print('trainset size:' ,trainset.__len__())
print('valset size:',valset.__len__())


trainset size: 19516
valset size: 1245


In [33]:


sample_idx = 10

#id tensors
tokens_tensor, segments_tensor, label_tensor,origin_text = trainset[sample_idx]

# 将 tokens_tensor 还原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())

label_map = {0: 'reliable', 1: 'unreliable'}

print('token:\n',tokens,'\n')
print('origin_text:\n',origin_text,'\n')
print('label:',label_map[int(label_tensor.numpy())],'\n')
print('tokens_tensor:\n',tokens_tensor,'\n')
print('segments_tensor:\n',segments_tensor,'\n')

token:
 ['[CLS]', 'want', 'to', 'get', 'this', 'briefing', 'by', 'email', 'here', 'the', 'good', 'evening', 'here', 'the', 'latest', 'the', 'meeting', 'between', 'tech', 'executives', 'and', 'donald', 'trump', 'included', 'leaders', 'of', 'some', 'of', 'the', 'world', 'most', 'successful', 'companies', 'including', 'jeff', 'be', '##zos', 'of', 'amazon', 'above', 'three', 'of', 'mr', 'trump', 'children', 'attended', 'what', 'unfolded', 'is', 'less', 'immediately', 'clear', 'the', 'press', 'was', 'quickly', 'ushered', 'out', 'of', 'the', 'room', 'mr', 'be', '##zos', 'had', 'already', 'made', 'news', 'announcing', 'that', 'amazon', 'made', 'its', 'first', 'commercial', 'drone', 'delivery', 'this', 'month', 'in', 'britain', 'deal', 'to', 'end', 'the', 'brutal', 'assaults', 'on', 'the', 'syrian', 'city', 'of', 'aleppo', 'fell', 'through', 'artillery', 'shell', '##ing', 'resumed', 'in', 'the', 'latest', 'bitter', 'turn', 'for', 'the', 'thousands', 'of', 'civilians', 'and', 'medical', 'worker

In [34]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

""""
create_mini_batch(samples)吃上面定義的mydataset
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

#限制attention只注意非pad 的部分
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
    
    # attention masks: 标注不为zero padding的部分
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids



# 初始化DataLoader：设置batch size

BATCH_SIZE = 16
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)
# testloader = DataLoader(testset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)

data = next(iter(trainloader))

In [35]:
tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")
tokens_tensors.shape 


tokens_tensors.shape   = torch.Size([16, 202]) 
tensor([[  101,  6221,  8398,  ...,     0,     0,     0],
        [  101,  2281,  3481,  ...,  1997,  1996,   102],
        [  101,  2023,  2695,  ...,  2031,  4942,   102],
        ...,
        [  101,  6643,  1996,  ...,  2014, 11368,   102],
        [  101, 21073,  7041,  ...,  2057,  5987,   102],
        [  101,  2006,  1996,  ...,  3679,  2003,   102]])
------------------------
segments_tensors.shape = torch.Size([16, 202])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
------------------------
masks_tensors.shape    = torch.Size([16, 202])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
---------

torch.Size([16, 202])

In [36]:
from transformers import BertForSequenceClassification

In [37]:
PRETRAINED_MODEL_NAME = "bert-base-uncased"

NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [38]:
print("""
name      module
--------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print("{:10}{}".format(name,n) )
    else:
        print("{:15} {}".format(name, module))


name      module
--------------------
bert      embeddings
bert      encoder
bert      pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [39]:
%%time
from sklearn.metrics import accuracy_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
EPOCHS = 6

for epoch in range(EPOCHS):
    correct = 0
    #total = 0
    train_loss , val_loss = 0.0 , 0.0
    train_acc, val_acc = 0, 0
    n, m = 0, 0
    model.train()
    for data in trainloader:
        n += 1
        tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]

    
        optimizer.zero_grad()
        
        # forward 
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        # outputs ： "(loss), logits, (hidden_states), (attentions)"
        
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        
        #get prediction and calulate acc
        logits = outputs[1]
        _, pred = torch.max(logits.data, 1)
        train_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())

        #  batch loss
        train_loss += loss.item()
    
    #validation
    with torch.no_grad():
        model.eval()
        for data in valloader:
            m += 1
            tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]
            val_outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
            
            logits = val_outputs[1]
            _, pred = torch.max(logits.data, 1)
            val_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())
            val_loss += val_outputs[0].item()

    print('[epoch %d] loss: %.4f, acc: %.4f, val loss: %4f, val acc: %4f' %
          (epoch+1, train_loss/n, train_acc/n, val_loss/m,  val_acc/m  ))

print('Done')

device: cuda:0
[epoch 1] loss: 0.1767, acc: 0.9299, val loss: 0.073285, val acc: 0.975160
[epoch 2] loss: 0.0638, acc: 0.9773, val loss: 0.066127, val acc: 0.974359
[epoch 3] loss: 0.0284, acc: 0.9906, val loss: 0.083928, val acc: 0.974359
[epoch 4] loss: 0.0156, acc: 0.9946, val loss: 0.072660, val acc: 0.984776
[epoch 5] loss: 0.0141, acc: 0.9950, val loss: 0.077864, val acc: 0.979167
[epoch 6] loss: 0.0108, acc: 0.9964, val loss: 0.102003, val acc: 0.979167
Done
CPU times: user 44min 14s, sys: 6.18 s, total: 44min 20s
Wall time: 44min 20s


In [ ]:
f = pd.read_csv(data_file, encoding='UTF-8',usecols=['text'])
test_data = f['text'].tolist()[:5000]

